# Load Library & Preprocess Data

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split

import posenet

from config import *
from Datagenerator.Dataloader import load_data
from Datagenerator.Datagenerator import *
from Utils.draw_utils import *
from Utils.evaluate import evaluate
from Decode.decode_utils import *
from Decode.Decoder import *
from Losses.loss import *

### Set Seed

In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

### Load Data 

In [3]:
df = load_data()
print(len(df))
df.head()

16816


,NAME,pelvis_X,pelvis_Y,r hip_X,r hip_Y,r knee_X,r knee_Y,r ankle_X,r ankle_Y,l hip_X,l hip_Y,l knee_X,l knee_Y,l ankle_X,l ankle_Y,spine_X,spine_Y,thorax_X,thorax_Y,upper neck_X,upper neck_Y,head top_X,head top_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,r shoulder_X,r shoulder_Y,r elbow_X,r elbow_Y,r wrist_X,r wrist_Y
0,../../Datasets/MPII/images/015601864.jpg,610,187,573,185,616,269,620,394,647,188,661,221,656,231,628,181,647,176,637.0201,189.8183,695.9799,108.1817,692,185,693,240,688,313,601,167,553,161,606,217
1,../../Datasets/MPII/images/015599452.jpg,763,568,806,543,-1,-1,-1,-1,720,593,-1,-1,-1,-1,723,429,683,290,682.0000,256.0000,676.0000,68.0000,719,299,711,516,545,466,647,281,555,410,563,296
2,../../Datasets/MPII/images/005808361.jpg,974,446,908,438,816,510,804,711,1040,454,906,528,883,707,979,349,985,253,982.7591,235.9694,962.2409,80.0306,1067,253,1167,353,1142,478,902,253,798,340,869,214
3,../../Datasets/MPII/images/086617615.jpg,248,341,201,340,305,375,301,461,294,342,335,370,331,455,263,302,279,263,277.0210,268.7786,305.9790,184.2214,314,264,327,320,362,346,244,261,260,335,328,354
4,../../Datasets/MPII/images/060111501.jpg,904,237,865,248,896,318,980,322,943,226,948,290,881,349,881,186,858,135,871.1877,180.4244,835.8123,58.5756,923,123,995,163,961,223,792,147,754,247,772,294


### Split train & test Data 

In [4]:
train_df, validation_df = train_test_split(df, test_size=0.15, random_state=SEED)

# Build Model

In [5]:
with tf.device('/device:GPU:0'):
    base_model = posenet.load_model(mode)
    inputs = tf.keras.Input(shape=(256,256,3))
    outputs = base_model(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile Model

### Compile 

In [6]:
model.compile(
     optimizer = optimizer,
    # loss = [tf.keras.losses.MeanAbsoluteError(),
    #        tf.keras.losses.MeanAbsoluteError()],
    # loss = [AdaptiveSwingLoss, criterion],
    # loss = [criterion2, criterion],
    loss = [criterion2, tf.keras.losses.MeanAbsoluteError()],
    # loss = criterion2,
    # loss = EMD_loss,
    # loss = criterion2,
    # loss = tf.keras.losses.MeanSquaredError(),
    loss_weights = loss_weight,
)

# Training 

In [7]:
train_gen = DataGenerator(
    list_IDs = train_df.index,
    df = df,
    batch_size = batch_size,
    shuffle = True,
    image_paths = train_df['NAME'],
    mode = 'fit',
)

val_gen = DataGenerator(
    list_IDs = validation_df.index,
    df = df,
    batch_size = batch_size,
    shuffle = False,
    image_paths = validation_df['NAME'],
    mode = 'fit',
)

ckpt = ModelCheckpoint(
    filepath = f'./checkpoints/{ckpt_path}.hdf5',
    # filepath = f'./checkpoints/{ckpt_path}.h5',
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    save_weights_only = True,
    mode = 'auto'
)

reducelr = ReduceLROnPlateau(
    monitor = 'loss',
    factor = .25,
    patience = 2,
    min_lr = 1e-5,
    verbose = 1,
)

In [ ]:
history = model.fit(
    train_gen,
    validation_data = val_gen,
    epochs = EPOCHS,
    callbacks = [ckpt],
    use_multiprocessing = False,
)

Epoch 1/100
1786/1786 [==============================] - ETA: 0s - loss: 4.6334 - mobile_net_v1_loss: 0.0033 - mobile_net_v1_1_loss: 1.3513
Epoch 00001: val_loss improved from inf to 4.39767, saving model to ./checkpoints/20201005_EverySports-K_verification_alpha-25.hdf5
1786/1786 [==============================] - 358s 200ms/step - loss: 4.6334 - mobile_net_v1_loss: 0.0033 - mobile_net_v1_1_loss: 1.3513 - val_loss: 4.3977 - val_mobile_net_v1_loss: 0.0031 - val_mobile_net_v1_1_loss: 1.3316
Epoch 2/100
1786/1786 [==============================] - ETA: 0s - loss: 4.3550 - mobile_net_v1_loss: 0.0030 - mobile_net_v1_1_loss: 1.3223
Epoch 00002: val_loss improved from 4.39767 to 4.30983, saving model to ./checkpoints/20201005_EverySports-K_verification_alpha-25.hdf5
1786/1786 [==============================] - 354s 198ms/step - loss: 4.3550 - mobile_net_v1_loss: 0.0030 - mobile_net_v1_1_loss: 1.3223 - val_loss: 4.3098 - val_mobile_net_v1_loss: 0.0030 - val_mobile_net_v1_1_loss: 1.3109
Epoch 

Epoch 17/100
1786/1786 [==============================] - ETA: 0s - loss: 3.9466 - mobile_net_v1_loss: 0.0027 - mobile_net_v1_1_loss: 1.2322
Epoch 00017: val_loss improved from 3.95711 to 3.94759, saving model to ./checkpoints/20201005_EverySports-K_verification_alpha-25.hdf5
1786/1786 [==============================] - 356s 199ms/step - loss: 3.9466 - mobile_net_v1_loss: 0.0027 - mobile_net_v1_1_loss: 1.2322 - val_loss: 3.9476 - val_mobile_net_v1_loss: 0.0027 - val_mobile_net_v1_1_loss: 1.2291
Epoch 18/100
1786/1786 [==============================] - ETA: 0s - loss: 3.9375 - mobile_net_v1_loss: 0.0027 - mobile_net_v1_1_loss: 1.2300
Epoch 00018: val_loss improved from 3.94759 to 3.93937, saving model to ./checkpoints/20201005_EverySports-K_verification_alpha-25.hdf5
1786/1786 [==============================] - 359s 201ms/step - loss: 3.9375 - mobile_net_v1_loss: 0.0027 - mobile_net_v1_1_loss: 1.2300 - val_loss: 3.9394 - val_mobile_net_v1_loss: 0.0027 - val_mobile_net_v1_1_loss: 1.2268


1786/1786 [==============================] - 355s 199ms/step - loss: 3.8384 - mobile_net_v1_loss: 0.0026 - mobile_net_v1_1_loss: 1.2107 - val_loss: 3.8555 - val_mobile_net_v1_loss: 0.0026 - val_mobile_net_v1_1_loss: 1.2080
Epoch 34/100
1786/1786 [==============================] - ETA: 0s - loss: 3.8333 - mobile_net_v1_loss: 0.0026 - mobile_net_v1_1_loss: 1.2099
Epoch 00034: val_loss did not improve from 3.85547
1786/1786 [==============================] - 354s 198ms/step - loss: 3.8333 - mobile_net_v1_loss: 0.0026 - mobile_net_v1_1_loss: 1.2099 - val_loss: 3.8582 - val_mobile_net_v1_loss: 0.0027 - val_mobile_net_v1_1_loss: 1.2073
Epoch 35/100
1786/1786 [==============================] - ETA: 0s - loss: 3.8283 - mobile_net_v1_loss: 0.0026 - mobile_net_v1_1_loss: 1.2091
Epoch 00035: val_loss improved from 3.85547 to 3.84831, saving model to ./checkpoints/20201005_EverySports-K_verification_alpha-25.hdf5
1786/1786 [==============================] - 355s 199ms/step - loss: 3.8283 - mobile_

Epoch 51/100
1786/1786 [==============================] - ETA: 0s - loss: 3.7603 - mobile_net_v1_loss: 0.0026 - mobile_net_v1_1_loss: 1.2002
Epoch 00051: val_loss improved from 3.80644 to 3.80488, saving model to ./checkpoints/20201005_EverySports-K_verification_alpha-25.hdf5
1786/1786 [==============================] - 354s 198ms/step - loss: 3.7603 - mobile_net_v1_loss: 0.0026 - mobile_net_v1_1_loss: 1.2002 - val_loss: 3.8049 - val_mobile_net_v1_loss: 0.0026 - val_mobile_net_v1_1_loss: 1.1977
Epoch 52/100
1786/1786 [==============================] - ETA: 0s - loss: 3.7567 - mobile_net_v1_loss: 0.0026 - mobile_net_v1_1_loss: 1.1998
Epoch 00052: val_loss improved from 3.80488 to 3.80171, saving model to ./checkpoints/20201005_EverySports-K_verification_alpha-25.hdf5
1786/1786 [==============================] - 356s 199ms/step - loss: 3.7567 - mobile_net_v1_loss: 0.0026 - mobile_net_v1_1_loss: 1.1998 - val_loss: 3.8017 - val_mobile_net_v1_loss: 0.0026 - val_mobile_net_v1_1_loss: 1.1972


Epoch 69/100
1268/1786 [====================>.........] - ETA: 1:27 - loss: 3.6951 - mobile_net_v1_loss: 0.0025 - mobile_net_v1_1_loss: 1.1980

In [ ]:
model.save_weights(f'./checkpoints/{ckpt_path}_final.h5')

# History 

In [ ]:
# draw_train_result(history)
plt.plot(history.history['loss'], 'y', label='train loss')
plt.plot(history.history['val_loss'], 'r', label='val loss')

best = min(history.history['val_loss'])
plt.title(f'Best Loss : {best}')

plt.xlabel('epoch')
plt.ylabel('loss')

plt.legend(loc='upper right')

# Inference 

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
with tf.device('/device:GPU:1'):
    base_model = posenet.load_model(mode)
    inputs = tf.keras.Input(shape=(256,256,3))
    outputs = base_model(inputs)
    inf_model = tf.keras.Model(inputs=inputs, outputs=outputs)
    # inf_model.load_weights(f'./checkpoints/{ckpt_path}.hdf5')
    inf_model.load_weights(f'./checkpoints/{ckpt_path}_final.h5')

In [ ]:
# decode_model = add_decoder(inf_model)

In [ ]:
inf_gen = EvalDataGenerator(
    list_IDs = validation_df.index,
    df = df,
    batch_size = 1,
    shuffle = False,
    image_paths = validation_df['NAME'],
    mode = 'fit',
)

In [ ]:
img, regr = inf_gen.__getitem__(11)
result = inf_model(img)
idx = 0

In [ ]:
plt.imshow(img[idx]/255.)
plt.show()

In [ ]:
draw_heatmap(0, *regr)

In [ ]:
draw_heatmap(0, heatmap_result=result[0])

# Decode 

In [ ]:
# pose_scores, keypoint_scores, keypoint_coords = decode_single_poses(result[0].numpy().squeeze(axis=0),
#                     result[1].numpy().squeeze(axis=0))

# print(f'pose score : {pose_scores}')
# print(f'keypoint scores :')
# print(f'{keypoint_scores}')
# print(f'keypoint coords :')
# print(f'{keypoint_coords}')

In [ ]:
pose_scores_nooff, keypoint_scores_nooff, keypoint_coords_nooff = decode_single_poses_nooff(result.numpy().squeeze(axis=0))

print(f'pose score nooff: {pose_scores_nooff}')
print(f'keypoint scores nooff:')
print(f'{keypoint_scores_nooff}')
print(f'keypoint coords nooff:')
print(f'{keypoint_coords_nooff}')

In [ ]:
draw_result(img, idx, keypoint_coords_nooff, keypoint_coords_nooff, result, inf_gen)

# Evaluate

In [ ]:
evaluate(inf_model, inf_gen)